# Engine Sizing
This code is meant to provide initial sizing estimates for the engine.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct  

# Defining Main Parameters
pc = 2.758e+6                # Chamber (Stagnation) pressure (Pa)
T_i = 298.15                # Propellant Inlet Temperature (K) = STP
thrust = 4448.22             # Desired thrust (N) = 1000 lbf 
p_amb = 101325               # Ambient pressure (Pa) - sea level pressure
OF = 9                      # OF mass ratio

# Defining exhaust gas parameters using Cantera CEA
gas = ct.Solution('gri30.yaml')
gas.TPY = 298.15, p0, "C2H6:1, N2O:{}".format(OF) 

# Running Cantera CEA
gas.equilibrate("HP")
Tc = gas.T                           # Combustion chamber temperature (K)
s0 = gas.s                           # Entropy - needed for isentropic expansion through nozzle
h0 = gas.h                           # Stagnation enthalpy 
gamma_c = gas.cp/gas.cv